# Welcome to ProgPy's State Limits Example Notebook!

In this notebook, we will demonstrate when and how to identify model state limits for use in state-transition models. We'll use a `ThrownObject` model to demonstrate the feature!


The `ThrownObject` model is a simple non-linear model that simulates an object being thrown into the air with air resistance taken into account. 

This model has two states:

1) `x` : The velocity in space __(meters per second)__
2) `v` : The position in space __(meters)__

Let's break down each state:

- __Position (x)__: This represents the current height of the object in space. It's measured in meters. When the object is first thrown, the position would be the height of the thrower. <span style="color:teal">As the object ascends and then descends, the position changes accordingly.<span>

- __Velocity (v)__: This represents the speed and direction of the object. It's measured in meters per second. When the object is first thrown, its velocity would be equal to the throwing speed.<span style="color:teal"> As the object rises and falls, the velocity changes. Specifically, while the object is rising and overcoming gravity, the velocity will decrease until it reaches zero at the peak of the trajectory. Then, as the object falls, the velocity will increase in the opposite direction.<span>

### Importing Modules

In [ ]:
from math import inf
from progpy.models.thrown_object import ThrownObject

First, we'll need to create an instance of our model. We'll be initalizing the model without any drag.

In [ ]:
m = ThrownObject(cd = 0)

Before we add state limits to our model, let's take a look at the results when there  are none!

In [ ]:
event = 'impact'
simulated_results = m.simulate_to_threshold(threshold_keys=[event], dt=0.005, save_freq=1)

print('Example: No State Limits')
for i, state in enumerate(simulated_results.states):
    print(f'State {i}: {state}')
print()

Notice that at the end of the simulation, the object's position is negative. Typically, when the `ThrownObject` object reaches the ground, the position simply be __0__. However, in this case, the position is negative. This is not a realistic result, so we'll need to add state limits to prevent this from happening.

This can be done by changing the `state_limits` attribute of the model. The `state_limits` attribute is a dictionary that contains the state limits for each state. The keys of the dictionary are the state names, and the values are tuples that contain the lower and upper limits of the state.

In [ ]:
m.state_limits = {
    # object may not go below ground height
    'x': (0, inf),

    # object may not exceed the speed of light
    'v': (-299792458, 299792458)
}

With our newly defined future loading function and our state limits defined, let's call the ProgPy's [Simulation](https://nasa.github.io/progpy/prog_models_guide.html#simulation) methods to simulate our model!

In [ ]:
event = 'impact'
simulated_results = m.simulate_to_threshold(threshold_keys=[event], dt=0.005, save_freq=1)

print('Example: With State Limits')
for i, state in enumerate(simulated_results.states):
    print(f'State {i}: {state}')
print()

Notice that our `x` position never reaches a negative value! This is because we have defined a state limit for the `x` state that prevents it from going below __0__ (see _Warning_ above). This makes our simulation more realistic!

Let's try another example, this time we'll try setting the `x` to a number outside of its bounds!

In [ ]:
x0 = m.initialize(u = {}, z = {})
x0['x'] = -1

simulated_results = m.simulate_to_threshold(threshold_keys=[event], dt=0.005, save_freq=1, x = x0)

# Print states
print('Example 2: With -1 as initial x value')
for i, state in enumerate(simulated_results.states):
    print('State ', i, ': ', state)
print()

Since our `x` state limit is set to __0__, the simulation will abruptly stop since our ThrownObject model goes is being thrown up.

Finally, let's try another example when the object speed approaches its limit.

In [ ]:
x0 = m.initialize(u = {}, z = {})
x0['x'] = 1000000000
x0['v'] = 0
m.parameters['g'] = -50000000

print('Example 3')
simulated_results = m.simulate_to_threshold(threshold_keys=[event], dt=0.005, save_freq=0.3, x = x0, print = True, progress = False)

# Note that the limits can also be applied manually using the apply_limits function
print('limiting states')
x = {'x': -5, 'v': 3e8}  # Too fast and below the ground
print('\t Pre-limit: {}'.format(x))
x = m.apply_limits(x)
print('\t Post-limit: {}'.format(x))

### Conclusion

Setting appropriate state limits is crucial in creating realistic and accurate state-transition models. It ensures that the model's behavior stays within realistic bounds, regardless of the conditions or variables applied. Keep in mind that the limits should be set based on the physical or practical constraints of the system being modeled.

For more information, please refer to our [State Limits](https://nasa.github.io/progpy/prog_models_guide.html#state-limits) documentation. For more information on ProgPy, please refer to our [ProgPy](https://nasa.github.io/progpy/) documentation.